### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Assigning data to variables
total_schools = len(school_data_complete["school_name"].unique())
total_students = (school_data_complete["student_name"].count())
total_budget = school_data["budget"].sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()
pass_math = 100*school_data_complete["math_score"].loc[(school_data_complete["math_score"]>=70)].count()/total_students
pass_read = 100*school_data_complete["reading_score"].loc[(school_data_complete["reading_score"]>=70)].count()/total_students
pass_overall = 100*school_data_complete["math_score"].loc[(school_data_complete["math_score"]>=70)&(school_data_complete["reading_score"]>=70)].count()/total_students

#Store in dataframe and format
district_summary_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":["{:,}".format(total_students)],
    "Total Budget":["${:,.2f}".format(total_budget)],
    "Average Math Score":[round(avg_math_score,2)],
    "Average Reading Score":[round(avg_read_score,2)],
    "% Passing Math":[round(pass_math,2)],
    "% Passing Reading":[round(pass_read,2)],
    "% Overall Passing":[round(pass_overall,2)]
})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#Add Passing columns (1 = pass, 0 = fail) for combined DF
school_data_complete["pass math"] = np.where(school_data_complete["math_score"]>=70,1,0)
school_data_complete["pass reading"] = np.where(school_data_complete["reading_score"]>=70,1,0)
school_data_complete["pass overall"] = np.where((school_data_complete["math_score"]>=70)&(school_data_complete["reading_score"]>=70),1,0)

#Get scores averages and pass % per school
scores_by_school = school_data_complete.groupby("school_name")[["reading_score","math_score","pass math","pass reading","pass overall"]].agg('mean')

#Merge with school data
school_summary = pd.merge(school_data, scores_by_school, how="left", on=["school_name", "school_name"])

#Add per student budget
school_summary["per_student_budget"] = school_summary["budget"]/school_summary["size"]

#Multiply pass by 100
school_summary["pass math"] = school_summary["pass math"]*100
school_summary["pass reading"] = school_summary["pass reading"]*100
school_summary["pass overall"] = school_summary["pass overall"]*100

#Filter columns and reorder
school_summary_clean = school_summary[["school_name","type","size","budget","per_student_budget","math_score","reading_score","pass math","pass reading","pass overall"]]

#Rename columns
school_summary_renamed = school_summary_clean.rename(columns={
    "school_name":" ",
    "type":"School Type",
    "size":"Total Students",
    "budget":"Total School Budget",
    "per_student_budget":"Per Student Budget",
    "math_score":"Average Math Score",
    "reading_score":"Average Reading Score",
    "pass math":"% Passing Math",
    "pass reading":"% Passing Reading",
    "pass overall":"% Overall Passing"
})

#Sort and format
school_summary_format = school_summary_renamed.sort_values(by=[" "]).style.format({
    'Total School Budget':'${:,.2f}',
    'Per Student Budget':'${:,.2f}',
})
school_summary_format.hide_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
#Sorting school_summary_renamed by Overall Passing
top_overall = school_summary_renamed.sort_values("% Overall Passing", ascending=False)

#Format and print
top_overall_format = top_overall.head().style.format({
    'Total School Budget':'${:,.2f}',
    'Per Student Budget':'${:,.2f}',
})
top_overall_format.hide_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
#Sorting school_summary_renamed by Overall Passing
bot_overall = school_summary_renamed.sort_values("% Overall Passing", ascending=True)

#Format and print
bot_overall_format = bot_overall.head().style.format({
    'Total School Budget':'${:,.2f}',
    'Per Student Budget':'${:,.2f}',
})
bot_overall_format.hide_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#Creating each series
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th",["school_name","math_score"]]
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th",["school_name","math_score"]]
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th",["school_name","math_score"]]
twelveth = school_data_complete.loc[school_data_complete["grade"] == "12th",["school_name","math_score"]]

#Grouping by school
ninth_grouped = ninth.groupby("school_name").mean()
tenth_grouped = tenth.groupby("school_name").mean()
eleventh_grouped = eleventh.groupby("school_name").mean()
twelveth_grouped = twelveth.groupby("school_name").mean()

#Merge and rename columns
ninth_tenth = pd.merge(ninth_grouped,tenth_grouped,how="left", on="school_name")
ninth_tenth_renamed = ninth_tenth.rename(columns={
    "math_score_x":"9th",
    "math_score_y":"10th"
})
ninth_tenth_eleventh = pd.merge(ninth_tenth_renamed,eleventh_grouped,how="left", on="school_name")
ninth_tenth_eleventh_twelveth = pd.merge(ninth_tenth_eleventh,twelveth_grouped,how="left", on="school_name")
math_bygrade = ninth_tenth_eleventh_twelveth.reset_index()
math_bygrade_renamed = math_bygrade.rename(columns={
    "school_name":"",
    "math_score_x":"11th",
    "math_score_y":"12th"
})
math_bygrade_renamed.style.hide_index()

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
#Creating series
read_9 = school_data_complete.loc[school_data_complete["grade"] == "9th",["school_name","reading_score"]]
read_10 = school_data_complete.loc[school_data_complete["grade"] == "10th",["school_name","reading_score"]]
read_11 = school_data_complete.loc[school_data_complete["grade"] == "11th",["school_name","reading_score"]]
read_12 = school_data_complete.loc[school_data_complete["grade"] == "12th",["school_name","reading_score"]]

#Grouping by school
read_9_grouped = read_9.groupby("school_name").mean()
read_10_grouped = read_10.groupby("school_name").mean()
read_11_grouped = read_11.groupby("school_name").mean()
read_12_grouped = read_12.groupby("school_name").mean()

#Merge and rename columns
read_9_10 = pd.merge(read_9_grouped,read_10_grouped,how="left", on="school_name")
read_9_10_renamed = read_9_10.rename(columns={
    "reading_score_x":"9th",
    "reading_score_y":"10th"
})
read_9_10_11 = pd.merge(read_9_10_renamed,read_11_grouped,how="left", on="school_name")
read_9_10_11_12 = pd.merge(read_9_10_11,read_12_grouped,how="left", on="school_name")
reading_bygrade = read_9_10_11_12.reset_index()
reading_bygrade_renamed = reading_bygrade.rename(columns={
    "school_name":"",
    "reading_score_x":"11th",
    "reading_score_y":"12th"
})
reading_bygrade_renamed.style.hide_index()

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
#Create bins
spending_bins = [0,585,630,645,680]
spending_categories = ["<$585","$585-630","$630-645","$645-680"]

#Copied school_data_complete and added Spending
spending_ranges = school_summary_renamed.copy()
spending_ranges["Spending Ranges (Per Student)"] = pd.cut(school_summary_renamed["Per Student Budget"],bins=spending_bins,labels=spending_categories,include_lowest=True)

#Groupby and round format
spending_ranges_grouped = spending_ranges.groupby("Spending Ranges (Per Student)")[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]].agg("mean")
spending_ranges_grouped.round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
#Create bins
size_bins = [0,1000,2000,5000]
size_categories = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

#Copied school_data_complete and added Spending
size_ranges = school_summary_renamed.copy()
size_ranges["School Size"] = pd.cut(school_summary_renamed["Total Students"],bins=size_bins,labels=size_categories,include_lowest=True)

#Groupby and print
size_ranges_grouped = size_ranges.groupby("School Size")[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]].agg("mean")
size_ranges_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [16]:
#Copied school_data_complete and Groupby School Type
school_type = school_summary_renamed.copy()
school_type_grouped = school_type.groupby("School Type")[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]].agg("mean")
school_type_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


## Conclusions

After analyzing the School and Student Data we can observe there are the following trends:

  * School Type & Size: There is a huge difference in Average Scores and % Passing, for Charter Schools 90% of students pass overall while only 54% of students pass in District Schools. When considering size, Small and Medium Schools have similar overall pass with 90% and only 58% for Large Schools. 
  
  
  * Spending Ranges: As Spending per Student increases the average scores and % passing decreases for every metric. 
  
We can conclude that in Small and Medium Schools students might be getting more personalized attention than in Large Schools and that might explain the better test scores and % of students that pass. On the other hand as School Spending per Student increases the Average Scores are dropping, this might be explained by having many distractions for students that lose focus on their study. 
  

